In [4]:
'''# 필요 라이브러리 설치
! pip install pytesseract
! pip install opencv-python-headless
! brew install tesseract-lang 
! pip install matplotlib
! pip install opencv-python
! pip install matplotlib
! pip install torch torchvision torchaudio

! git clone https://github.com/ultralytics/yolov5
! pip install -r /Users/kyungrim/fake-license-plate-detection/yolov5/requirements.txt'''


# 라이브러리 선언
import cv2
import numpy as np
import pandas as pd
import torch
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import os
import torch
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'


# YOLOv5 모델 로드
model_path = './best.pt'
model = torch.hub.load('./yolov5', 'custom', path=model_path, source='local')

YOLOv5 🚀 v7.0-388-g882c35fc Python-3.12.2 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [10]:
# 함수 선언

# 차량 진입 (차량 이미지 입력)
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8, resize_dim=(800, 600)):
    """
    파일 경로를 읽어 이미지를 로드하고, 크기를 조정합니다.
    :param filename: 이미지 파일 경로
    :param flags: OpenCV 이미지 로드 플래그 (기본값: cv2.IMREAD_COLOR)
    :param dtype: 데이터 타입 (기본값: np.uint8)
    :param resize_dim: 이미지 크기 조정 (너비, 높이)
    :return: 로드된 이미지 또는 None
    """
    try:
        n = np.fromfile(filename, dtype)  # 한글 경로 지원을 위해 np.fromfile 사용
        img = cv2.imdecode(n, flags)     # 이미지 디코딩
        if img is not None and resize_dim:  # 이미지가 로드되었고 크기 조정 설정이 있을 때
            img = cv2.resize(img, resize_dim)
        return img
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        return None


# 1. 번호판 인식 & 비교

MAX_DIAG_MULTIPLYER = 5
MAX_ANGLE_DIFF = 12.0
MAX_AREA_DIFF = 0.5
MAX_WIDTH_DIFF = 0.8
MAX_HEIGHT_DIFF = 0.2
MIN_N_MATCHED = 3

def img_to_gray(image, image_show=True):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

def detecting_car_plate(img) :
    # YOLOv5로 객체 탐지
    results = model(img)

    detections = results.xyxy[0]  # 탐지된 객체의 좌표
    for i, (*box, conf, cls) in enumerate(detections):
        if int(cls) == 1:  # 번호판 클래스만 처리
            x1, y1, x2, y2 = map(int, box)  # 바운딩 박스 좌표
            cropped_plate = img[y1:y2, x1:x2]  # 번호판 영역 자르기

    return cropped_plate
    
def extracting_car_number(img) :
    # YOLOv5로 객체 탐지
    img_ori = detecting_car_plate(img)

    height, width, channel = img_ori.shape
    channel = 1

    # Morphology Operation
    StructuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    topHat = cv2.morphologyEx(img_ori, cv2.MORPH_TOPHAT, StructuringElement)
    blackHat = cv2.morphologyEx(img_ori, cv2.MORPH_BLACKHAT, StructuringElement)
    img_topHat = cv2.add(img_ori, topHat)
    img_ori = cv2.subtract(img_topHat, blackHat)
    gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
    # Gaussian Blurring
    blur = cv2.GaussianBlur(gray, (5, 5), 2)

    # Adaptive Thresholding
    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    img_blur_thresh = cv2.adaptiveThreshold(
    img_blurred,
    maxValue=255.0,
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV,
    blockSize=19,
    C=9
    )

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    img_blur_thresh = cv2.adaptiveThreshold(
        img_blurred,
        maxValue=255.0,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=cv2.THRESH_BINARY_INV,
        blockSize=19,
        C=11
    )

    contours, _ = cv2.findContours(
        img_blur_thresh,
        mode=cv2.RETR_LIST,
        method=cv2.CHAIN_APPROX_SIMPLE
    )

    temp_result = np.zeros((height, width, channel), dtype=np.uint8)

    cv2.drawContours(temp_result, contours=contours, contourIdx=-1, color=(255,255,255))

    temp_result = np.zeros((height, width, channel), dtype=np.uint8)

    contours_dict = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(temp_result, pt1=(x,y), pt2=(x+w, y+h), color=(255,255,255), thickness=2)
        
        contours_dict.append({
            'contour': contour,
            'x': x,
            'y': y,
            'w': w,
            'h': h,
            'cx': x + (w / 2),
            'cy': y + (h / 2)
        })

    MIN_AREA = 80
    MIN_WIDTH, MIN_HEIGHT=2, 8
    MIN_RATIO, MAX_RATIO = 0.2, 1.0

    possible_contours = []

    cnt = 0
    for d in contours_dict:
        area = d['w'] * d['h']
        ratio = d['w'] / d['h']
        
        if area > MIN_AREA \
        and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT \
        and MIN_RATIO < ratio < MAX_RATIO:
            d['idx'] = cnt
            cnt += 1
            possible_contours.append(d)

    temp_result = np.zeros((height, width, channel), dtype = np.uint8)

    for d in possible_contours:
        cv2.rectangle(temp_result, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255, 255, 255), thickness=2)

    def find_chars(contour_list):
        matched_result_idx = []
        
        for d1 in contour_list:
            matched_contours_idx = []
            for d2 in contour_list:
                if d1['idx'] == d2['idx']:
                    continue
                    
                dx = abs(d1['cx'] - d2['cx'])
                dy = abs(d1['cy'] - d2['cy'])
                
                diagonal_length1 = np.sqrt(d1['w'] ** 2 + d1['h'] ** 2)
                
                distance = np.linalg.norm(np.array([d1['cx'], d1['cy']]) - np.array([d2['cx'], d2['cy']]))
                if dx == 0:
                    angle_diff = 90
                else:
                    angle_diff = np.degrees(np.arctan(dy / dx))
                area_diff = abs(d1['w'] * d1['h'] - d2['w'] * d2['h']) / (d1['w'] * d1['h'])
                width_diff = abs(d1['w'] - d2['w']) / d1['w']
                height_diff = abs(d1['h'] - d2['h']) / d1['h']
                
                if distance < diagonal_length1 * MAX_DIAG_MULTIPLYER \
                and angle_diff < MAX_ANGLE_DIFF and area_diff < MAX_AREA_DIFF \
                and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                    matched_contours_idx.append(d2['idx'])
                    
            matched_contours_idx.append(d1['idx'])
            
            if len(matched_contours_idx) < MIN_N_MATCHED:
                continue
                
            matched_result_idx.append(matched_contours_idx)
            
            unmatched_contour_idx = []
            for d4 in contour_list:
                if d4['idx'] not in matched_contours_idx:
                    unmatched_contour_idx.append(d4['idx'])
            
            unmatched_contour = np.take(possible_contours, unmatched_contour_idx)
            
            recursive_contour_list = find_chars(unmatched_contour)
            
            for idx in recursive_contour_list:
                matched_result_idx.append(idx)
                
            break
            
        return matched_result_idx

    result_idx = find_chars(possible_contours)

    matched_result = []
    for idx_list in result_idx:
        matched_result.append(np.take(possible_contours, idx_list))
        
    temp_result = np.zeros((height, width, channel), dtype=np.uint8)

    for r in matched_result:
        for d in r:
            cv2.rectangle(temp_result, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255,255,255), thickness=2)

    PLATE_WIDTH_PADDING = 1.3 # 1.3
    PLATE_HEIGHT_PADDING = 1.5 # 1.5
    MIN_PLATE_RATIO = 3
    MAX_PLATE_RATIO = 10

    plate_imgs = []
    plate_infos = []

    for i, matched_chars in enumerate(matched_result):
        sorted_chars = sorted(matched_chars, key=lambda x: x['cx'])

        plate_cx = (sorted_chars[0]['cx'] + sorted_chars[-1]['cx']) / 2
        plate_cy = (sorted_chars[0]['cy'] + sorted_chars[-1]['cy']) / 2
        
        plate_width = (sorted_chars[-1]['x'] + sorted_chars[-1]['w'] - sorted_chars[0]['x']) * PLATE_WIDTH_PADDING
        
        sum_height = 0
        for d in sorted_chars:
            sum_height += d['h']

        plate_height = int(sum_height / len(sorted_chars) * PLATE_HEIGHT_PADDING)
        
        triangle_height = sorted_chars[-1]['cy'] - sorted_chars[0]['cy']
        triangle_hypotenus = np.linalg.norm(
            np.array([sorted_chars[0]['cx'], sorted_chars[0]['cy']]) - 
            np.array([sorted_chars[-1]['cx'], sorted_chars[-1]['cy']])
        )
        
        angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus))
        
        rotation_matrix = cv2.getRotationMatrix2D(center=(plate_cx, plate_cy), angle=angle, scale=1.0)
        
        img_rotated = cv2.warpAffine(img_blur_thresh, M=rotation_matrix, dsize=(width, height))
        
        img_cropped = cv2.getRectSubPix(
            img_rotated, 
            patchSize=(int(plate_width), int(plate_height)), 
            center=(int(plate_cx), int(plate_cy))
        )
        
        if img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO or img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO > MAX_PLATE_RATIO:
            continue
        
        plate_imgs.append(img_cropped)
        plate_infos.append({
            'x': int(plate_cx - plate_width / 2),
            'y': int(plate_cy - plate_height / 2),
            'w': int(plate_width),
            'h': int(plate_height)
        })

    longest_idx, longest_text = -1, 0
    plate_chars = []

    for i, plate_img in enumerate(plate_imgs):
        plate_img = cv2.resize(plate_img, dsize=(0, 0), fx=1.6, fy=1.6)
        _, plate_img = cv2.threshold(plate_img, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        # find contours again (same as above)
        contours, _ = cv2.findContours(plate_img, mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_SIMPLE)
        
        plate_min_x, plate_min_y = plate_img.shape[1], plate_img.shape[0]
        plate_max_x, plate_max_y = 0, 0

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            
            area = w * h
            ratio = w / h

            if area > MIN_AREA \
            and w > MIN_WIDTH and h > MIN_HEIGHT \
            and MIN_RATIO < ratio < MAX_RATIO:
                if x < plate_min_x:
                    plate_min_x = x
                if y < plate_min_y:
                    plate_min_y = y
                if x + w > plate_max_x:
                    plate_max_x = x + w
                if y + h > plate_max_y:
                    plate_max_y = y + h
                    
        img_result = plate_img[plate_min_y:plate_max_y, plate_min_x:plate_max_x]
        
        img_result = cv2.GaussianBlur(img_result, ksize=(3, 3), sigmaX=0)
        _, img_result = cv2.threshold(img_result, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        img_result = cv2.copyMakeBorder(img_result, top=10, bottom=10, left=10, right=10, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))
        
        chars = pytesseract.image_to_string(img_result, lang='kornum+kor', config='--psm 6 preserve_interword_spaces')
        #chars = pytesseract.image_to_string(img_result, lang='kornum+kor', config='--psm 7 --oem 3')
        
        result_chars = ''
        has_digit = False
        for c in chars:
            if ord('가') <= ord(c) <= ord('힣') or c.isdigit():
                if c.isdigit():
                    has_digit = True
                result_chars += c
        
        plate_chars.append(result_chars)

        if has_digit and len(result_chars) > longest_text:
            longest_idx = i

    return result_chars

def is_car_number_in_database(extracted_number, database_numbers):
    return any(extracted_number in db_number or db_number in extracted_number for db_number in database_numbers)

# 유사 번호판 찾기 함수
def find_closest_plate(entering_number, database_numbers):
    # 입력된 번호판과 데이터베이스의 번호판을 비교하여 가장 유사한 번호판을 반환
    for db_number in database_numbers:
        if entering_number in db_number or db_number in entering_number:
            return db_number
    return None


# 2. 이미지 매칭 & 비교

def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (600, 300))
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

def hamming_distance(a, b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    # 같은 자리의 값이 서로 다른 것들의 합
    distance = (a !=b).sum()
    return distance

matching_criterion = 250
desc_criterion = 70
def image_match(image, stored_hash):
    # 들어온 차량의 이미지
    income_car_image = image    
    # 차 영상의 해쉬 구하기
    query_hash = img2hash(income_car_image)
    
    # 해당 차량의 저장된 해쉬 불러오기
    stored_hash = stored_hash
    
    # 해밍 거리 산출
    flag = 0
    hamming_dst = hamming_distance(query_hash, stored_hash)
    if hamming_dst/256 < matching_criterion: flag=1; print(f'  ···이미지 매칭량 : {hamming_dst/256}'); print('  ···동일 차량입니다.')
    else: print(f'  ···이미지 매칭량 : {hamming_dst/256}'); print('  ···동일하지 않은 차량입니다.')
        
    return income_car_image, flag



# 3. 특징점 매칭 & 비교

# ORB로 서술자 추출 
detector = cv2.ORB_create()
# BF-Hamming 생성
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)

def get_desc(img):
    img = img
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, desc = detector.detectAndCompute(gray, None)
    return kp, desc

def ORB_knnMatch(income_car_image, flag, stored_kp, stored_desc):
    # 각 영상에 대해 키 포인트와 서술자 추출 
    compare_kp, compare_desc = get_desc(income_car_image)
    # knnMatch, k=2
    matches = matcher.knnMatch(compare_desc, stored_desc, 2)

    # 첫번재 이웃의 거리가 두 번째 이웃 거리의 75% 이내인 것만 추출---⑤
    ratio = 0.75; desc_flag = 0
    good_matches = [first for first,second in matches \
                        if first.distance < second.distance * ratio]
    if len(good_matches) > desc_criterion: desc_flag = 1; print('  ···동일 차량입니다.')
    else: print('  ···동일하지 않은 차량입니다')
    print(f'  ···특징점 매칭량 : {len(good_matches)}/{len(matches)}')
    return desc_flag


# 4. 내부 존재 여부 확인
car1 = cv2.imread('/Users/kyungrim/fake-license-plate-detection/test_car/lightcar1.jpg')
car2 = cv2.imread('/Users/kyungrim/fake-license-plate-detection/test_car/whitecar1.jpg')

car1_kp, car1_desc = get_desc(car1)
car2_kp, car2_desc = get_desc(car2)

# 데이터베이스 (예시)
database = pd.DataFrame({
    '번호판' : [extracting_car_number(car1), extracting_car_number(car2)],
    'StoredHash' : [img2hash(car1), img2hash(car2)],
    'KeyPoints' : [car1_kp, car2_kp],
    'Descriptors' : [car1_desc, car2_desc],
    '내부존재여부' : [0, 0]
})


database

/Users/kyungrim/fake-license-plate-detection/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/kyungrim/fake-license-plate-detection/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


,번호판,StoredHash,KeyPoints,Descriptors,내부존재여부
0,157모4876,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(< cv2.KeyPoint 0x178851080>, < cv2.KeyPoint 0...","[[165, 20, 24, 216, 237, 181, 68, 91, 85, 138,...",0
1,05루7320,"[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","(< cv2.KeyPoint 0x1761387e0>, < cv2.KeyPoint 0...","[[166, 223, 103, 75, 202, 53, 175, 252, 141, 3...",0


In [11]:
class Car :
    def __init__(self, img) :
        self.car_number = extracting_car_number(img)
        self.hash = img2hash(img)
        self.kp, self.desc = get_desc(img)

In [12]:
# 데코레이터 정의
def security_check(step_name):
    def decorator(func):
        def wrapper(*args, **kwargs):
            global cnt  # 전역 변수 cnt 접근
            print(f"검사 시작: {step_name}")
            result = func(*args, **kwargs)
            if result:
                print(f"통과: {step_name}")
                cnt += 1
            else:
                print(f"실패: {step_name}")
            return result
        return wrapper
    return decorator

# 보안 검사 코드에 데코레이터 적용
@security_check("번호판 등록 여부 확인")
def check_car_registration(car_number, database):
    return is_car_number_in_database(car_number, database['번호판'].tolist())

@security_check("이미지 매칭 확인")
def check_image_match(car_image, stored_hash):
    _, flag = image_match(car_image, stored_hash)
    return flag == 1

@security_check("특징점 매칭 확인")
def check_feature_match(car_image, stored_kp, stored_desc):
    return ORB_knnMatch(car_image, 1, stored_kp, stored_desc)  # flag는 1로 설정

@security_check("내부 존재 여부 확인")
def check_internal_presence(closest_plate, database):
    return database.loc[database['번호판'] == closest_plate, '내부존재여부'].values[0] == 0

In [13]:
print("진입한 차량의 사진을 업로드 해주세요.")
car_image = imread('/Users/kyungrim/fake-license-plate-detection/test_car/lightcar2.jpg')
entering_car = Car(car_image)
cnt = 0  # 통과 단계 count

# 번호판 인식
cropped_plate = detecting_car_plate(car_image)
car_number = entering_car.car_number
if car_number:
    print("진입한 차량의 번호판:", car_number)

# 1. 번호판 등록 여부 확인
check_car_registration(car_number, database)

# 가장 유사한 번호판 찾기
closest_plate = find_closest_plate(car_number, database['번호판'].tolist())

# 2. 이미지 매칭 확인
stored_hash = database.loc[database['번호판'] == closest_plate, 'StoredHash'].values[0]
check_image_match(car_image, stored_hash)

# 3. 특징점 매칭 확인
stored_kp = database.loc[database['번호판'] == closest_plate, 'KeyPoints'].values[0]
stored_desc = database.loc[database['번호판'] == closest_plate, 'Descriptors'].values[0]
check_feature_match(car_image, stored_kp, stored_desc)

# 4. 내부 존재 여부 확인
check_internal_presence(closest_plate, database)

# 최종 진입 여부 판단
if cnt == 4:
    print("🚘 모든 보안을 통과했습니다. 진입하십시오. 🚘")
else:
    print("❌ 보안을 통과하지 못했습니다. 출차하십시오. ❌")


진입한 차량의 사진을 업로드 해주세요.


/Users/kyungrim/fake-license-plate-detection/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


진입한 차량의 번호판: 157모48761
검사 시작: 번호판 등록 여부 확인
통과: 번호판 등록 여부 확인
검사 시작: 이미지 매칭 확인
  ···이미지 매칭량 : 205.078125
  ···동일 차량입니다.
통과: 이미지 매칭 확인
검사 시작: 특징점 매칭 확인
  ···동일 차량입니다.
  ···특징점 매칭량 : 94/500
통과: 특징점 매칭 확인
검사 시작: 내부 존재 여부 확인
통과: 내부 존재 여부 확인
🚘 모든 보안을 통과했습니다. 진입하십시오. 🚘


/Users/kyungrim/fake-license-plate-detection/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
